##A Multi-Agent AI Research Assistant Using CrewAI and Gemini LLM

In [2]:
!pip install crewai
!pip install arxiv
!pip install pymupdf
!pip install -U langchain langchain-community duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:

In [3]:
# ✅ Core imports
import os
from crewai import LLM
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import Tool
from crewai.tools import BaseTool


In [4]:
# ✅ Set Gemini API key
os.environ["GEMINI_API_KEY"] = "your-gemini-api-key"

# ✅ Create Gemini LLM
llm = LLM(
    model="gemini/gemini-1.5-pro-latest",
    temperature=0.7,
)

In [5]:
class MyCustomDuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web for a given query."

    def _run(self, query: str) -> str:
        duckduckgo_tool = DuckDuckGoSearchRun()
        response = duckduckgo_tool.invoke(query)

        return response

In [6]:
# Load an AI paper from arXiv
arxiv_query = "efficient transformer architecture"
loader = ArxivLoader(query=arxiv_query, load_max_docs=1)
docs = loader.load()

# Split long docs
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(docs)
paper_text = split_docs[0].page_content if split_docs else "No paper found."

In [7]:
# 🧠 Agents
fetcher = Agent(
    role="Research Paper Finder",
    goal="Find the most recent impactful research papers",
    backstory="Expert in AI literature and research trends.",
    tools=[MyCustomDuckDuckGoTool()],
    verbose=True,
    llm=llm
)

summarizer = Agent(
    role="Research Summarizer",
    goal="Summarize AI/ML research in a clear and concise way",
    backstory="PhD in machine learning with a knack for simplifying technical papers.",
    verbose=True,
    llm=llm
)

critic = Agent(
    role="Research Critic",
    goal="Analyze and critique research papers for impact, novelty, and weaknesses",
    backstory="An AI ethics and research evaluation expert who provides balanced reviews.",
    verbose=True,
    llm=llm
)

publisher = Agent(
    role="Content Publisher",
    goal="Format summaries and critiques into a blog post",
    backstory="An editorial AI with great taste for markdown and newsletters.",
    verbose=True,
    llm=llm
)


In [8]:
# 🧠 Tasks
summarize_task = Task(
    description=f"Summarize the research paper below in bullet points:\n\n{paper_text}",
    expected_output="A concise bullet point summary of the paper.",
    agent=summarizer
)

critique_task = Task(
    description=f"Critically evaluate the following paper:\n\n{paper_text}\n\nFocus on: novelty, real-world application, limitations.",
    expected_output="A paragraph highlighting strengths, weaknesses, and impact.",
    agent=critic
)

publish_task = Task(
    description="Combine the summary and critique into a markdown blog post format. Include title, bullets, and critic comments.",
    expected_output="Markdown formatted blog post for the research digest.",
    agent=publisher
)


In [9]:
# 🧠 Crew Assembly
crew = Crew(
    agents=[summarizer, critic, publisher],
    tasks=[summarize_task, critique_task, publish_task],
    verbose=True
)

result = crew.kickoff()
print("\n\n📄 Final Blog Post:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 33ace384-a386-4fe6-a44c-e8e41418288a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 3df21509-6ef5-4ece-9844-af9cef3111e7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 3df21509-6ef5-4ece-9844-af9cef3111e7
       Status: Executing Task...
    └── 🤖 Agent: Research Summarizer
            Status: In Progress

# Agent: Research Summarizer
## Task: Summarize the research paper below in bullet points:

Neural Architecture Search on Efﬁcient Transformers and Beyond
Zexiang Liu1, Dong Li1, Kaiyue Lu1, Zhen Qin1, Weixuan Sun3,1, Jiacheng Xu1, Yiran Zhong1,2*
1SenseTime Research
2Shanghai AI Lab
3Australian National University
Abstract
Recently, numerous efﬁcient Transformers have been
proposed to reduce the quadratic computational complex-
ity of standard Transformers caused by the Softmax atten-
tion. However, most of them simply swap Softmax with an
efﬁcient attention mechanism without considering the cus-
tomized architectures specially for the efﬁcient attention.
In this paper, we argue that the handcrafted vanilla Trans-
former architectures for Softmax attention may not be suit-
able for efﬁcient Transformers. To address this issue, we
propose a new framework to ﬁnd optimal architectures for
efﬁcient Transformers with the neural architecture search
(NAS) technique. The proposed method is va

🤖 Agent: Research Summarizer
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 3df21509-6ef5-4ece-9844-af9cef3111e7
│      Assigned to: Research Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Summarizer
│           Status: ✅ Completed
├── 📋 Task: e7d4381f-0008-45b3-b0f9-46581d40475e
│      Assigned to: Research Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Critic
│           Status: ✅ Completed
└── 📋 Task: dd6b5c58-a7b6-4674-ab25-fa77bfef51b3
       Status: Executing Task...
    └── 🤖 Agent: Content Publisher
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 3df21509-6ef5-4ece-9844-af9cef3111e7
│      Assigned to: Research Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Summarizer
│           Status: ✅ Completed
├── 📋 Task: e7d4381f-0008-45b3-b0f9-46581d40475e
│      Assigned to: Research Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Critic
│           Status: ✅ Completed
└── 📋 Task: dd6b5c58-a7b6-4674-ab25-fa77bfef51b3
       Assigned to: Content Publisher
       Status: ✅ Completed
    └── 🤖 Agent: Content Publisher
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: dd6b5c58-a7b6-4674-ab25-fa77bfef51b3                                                                     │
│  Agent: Content Publisher                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 33ace384-a386-4fe6-a44c-e8e41418288a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



📄 Final Blog Post:
 ```markdown
# Co-Designing Architectures for Efficient Transformers: A Neural Architecture Search Approach

**Introduction:**

This blog post discusses a research paper that explores the use of Neural Architecture Search (NAS) to optimize the architecture of efficient Transformers.  The authors argue that simply replacing the Softmax attention mechanism with more efficient alternatives isn't enough for optimal performance.  They propose that the architecture itself should be co-designed with the attention mechanism.

**Summary:**

* **Problem:** Existing efficient Transformers often use new attention mechanisms within standard Transformer architectures, which may not be ideal.
* **Solution:** Employ NAS to automatically discover optimal architectures tailored for specific efficient attention mechanisms.
* **Method:** A new NAS framework explores and selects the best-performing architecture configurations.
* **Evaluation:**  Performance is evaluated on machine tran